<center><img src="env.png" height="600" width="600" /></center>

<center><img src="prob.png" height="600" width="600" /></center>

In [1]:
from pprint import pprint

In [2]:
states = []
for x in range(4):
    for y in range(4):
        states.append((x,y))
        
pprint(states)

[(0, 0),
 (0, 1),
 (0, 2),
 (0, 3),
 (1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (3, 0),
 (3, 1),
 (3, 2),
 (3, 3)]


In [3]:
actions = {"R": (0,1), "L": (0,-1), "U": (-1,0),"D": (1,0)}

In [5]:
rewards = {"({} {})".format(*s): -1 for s in states}

In [7]:
rewards["(0 1)"] = -2
rewards["(1 1)"] = -2
rewards["(3 0)"] = -2
rewards["(0 3)"] = -2
rewards["(1 3)"] = -2
rewards["(3 2)"] = -2
rewards["(3 3)"] = 10

pprint(rewards)

{'(0 0)': -1,
 '(0 1)': -2,
 '(0 2)': -1,
 '(0 3)': -2,
 '(1 0)': -1,
 '(1 1)': -2,
 '(1 2)': -1,
 '(1 3)': -2,
 '(2 0)': -1,
 '(2 1)': -1,
 '(2 2)': -1,
 '(2 3)': -1,
 '(3 0)': -2,
 '(3 1)': -1,
 '(3 2)': -2,
 '(3 3)': 10}


In [11]:
transitions = {"({} {})".format(*state):{} for state in states}

In [12]:
pprint(transitions)

{'(0 0)': {},
 '(0 1)': {},
 '(0 2)': {},
 '(0 3)': {},
 '(1 0)': {},
 '(1 1)': {},
 '(1 2)': {},
 '(1 3)': {},
 '(2 0)': {},
 '(2 1)': {},
 '(2 2)': {},
 '(2 3)': {},
 '(3 0)': {},
 '(3 1)': {},
 '(3 2)': {},
 '(3 3)': {}}


In [15]:

def build_action(state, prob):
    return [prob, "({} {})".format(*state)]


"""
format of the trasitions
(x y) = {"R" [prob, (x1 y1), "L" [prob, (x1 y1), "D" [prob, (x1 y1)] ....}
"""

for state in states:
    for action in actions:
        count=0.0
        result = tuple(( sum(x) for x in zip(state,actions[action])))
        #transitions
        T = []
        if result in states:
            T.append(build_action(result, 0.0))
            T.append(build_action(state, 0.1))
            count += 0.1
            
        else:
            T.append(build_action(state,0.0))
        
        for a in actions:
            if a != action:
                result2 = tuple(( sum(x) for x in zip(state,actions[a])))
                if result2 in states:
                    count+=0.1
                    T.append(build_action(result2,0.1))
        T[0][0] = 1.0 - count
        
        transitions["({} {})".format(*state)][action] = T

transitions["(3 3)"] = {"EXIT":[(0.0, "(3 3)")]}

pprint(transitions)
                    

{'(0 0)': {'D': [[0.8, '(1 0)'], [0.1, '(0 0)'], [0.1, '(0 1)']],
           'L': [[0.8, '(0 0)'], [0.1, '(0 1)'], [0.1, '(1 0)']],
           'R': [[0.8, '(0 1)'], [0.1, '(0 0)'], [0.1, '(1 0)']],
           'U': [[0.8, '(0 0)'], [0.1, '(0 1)'], [0.1, '(1 0)']]},
 '(0 1)': {'D': [[0.7, '(1 1)'],
                 [0.1, '(0 1)'],
                 [0.1, '(0 2)'],
                 [0.1, '(0 0)']],
           'L': [[0.7, '(0 0)'],
                 [0.1, '(0 1)'],
                 [0.1, '(0 2)'],
                 [0.1, '(1 1)']],
           'R': [[0.7, '(0 2)'],
                 [0.1, '(0 1)'],
                 [0.1, '(0 0)'],
                 [0.1, '(1 1)']],
           'U': [[0.7, '(0 1)'],
                 [0.1, '(0 2)'],
                 [0.1, '(0 0)'],
                 [0.1, '(1 1)']]},
 '(0 2)': {'D': [[0.7, '(1 2)'],
                 [0.1, '(0 2)'],
                 [0.1, '(0 3)'],
                 [0.1, '(0 1)']],
           'L': [[0.7, '(0 1)'],
                 [0.1, '(0 2)'],
   

In [20]:
class mdp:
    def __init__(self, transitions, rewards):
        self.transitions = transitions
        self.rewards = rewards
        self.states = transitions.keys()
        
        
    def get_rewards(self, state):
        return self.rewards[state]
    
    def get_actions(self, state):
        return self.transitions[state].keys()
    
    def get_transitions(self, state, action):
        return self.transitions[state][action]

In [21]:
environment = mdp(transitions, rewards)

<center><img src="bellman.svg" height="600" width="600" /></center>

In [37]:
def value_iteration(env):
    states= env.states
    actions= env.get_actions
    T = env.get_transitions
    R = env.get_rewards
    
    gamma = 0.9
    ep = 0.00001
    
    
    
    
    V1 = {s:0 for s in states}
    
    while True:
        V = V1.copy()
        delta = 0 
        
        
        for s in states:
            V1[s] = R(s) + gamma * max([ sum([p * V[s1] for (p, s1) in T(s, a)]) for a in actions(s)])
            delta = max(delta, abs(V1[s] - V[s]))
            
        if delta < ep:
            return V

In [38]:
pprint(value_iteration(environment))

{'(0 0)': -3.092719388835238,
 '(0 1)': -3.3380410053807728,
 '(0 2)': -0.9018350642021,
 '(0 3)': -0.5044671603107349,
 '(1 0)': -2.1026969658622923,
 '(1 1)': -2.123442942974675,
 '(1 2)': 0.833587273384276,
 '(1 3)': 2.2529194509655506,
 '(2 0)': -0.7047508166535572,
 '(2 1)': 0.8931765818020474,
 '(2 2)': 3.3853320941711322,
 '(2 3)': 6.3818047157841855,
 '(3 0)': -0.16419563358594003,
 '(3 1)': 2.658349321849511,
 '(3 2)': 5.323001076641234,
 '(3 3)': 10.0}


In [40]:
def policy(env):
    states= env.states
    actions= env.get_actions
    T = env.get_transitions
    V = value_iteration(env)
    
    
    P={}
    for s in states:
        P[s] = max(actions(s),  key= lambda a:sum([p*V[s1]  for (p,s1) in T(s,a)]))
    return P

In [41]:
pprint(policy(environment))

{'(0 0)': 'D',
 '(0 1)': 'R',
 '(0 2)': 'D',
 '(0 3)': 'D',
 '(1 0)': 'D',
 '(1 1)': 'D',
 '(1 2)': 'D',
 '(1 3)': 'D',
 '(2 0)': 'R',
 '(2 1)': 'R',
 '(2 2)': 'R',
 '(2 3)': 'D',
 '(3 0)': 'R',
 '(3 1)': 'R',
 '(3 2)': 'R',
 '(3 3)': 'EXIT'}


<center><img src="env.png" height="600" width="600" /></center>